In [1]:
%matplotlib

import math 
import pywt 
import numpy as np
import os
import csv
import fnmatch
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

Using matplotlib backend: Qt5Agg


In [2]:
TimeDwellOrig = 800
TimeFixation = 300

In [3]:
def filterBlinksNew(pupilData, timeListComplete):
    # filter any blinks and nan values lasting around 250ms (on average)
    # http://faculty.washington.edu/chudler/facts.html
   
    
   # Blink detection    
    missingPupilData = dict()
    missingPupilData['Start'] = list()
    missingPupilData['End'] = list()
    missingPupilData['Duration'] = list()
    missingPupilData['DurationExtra'] = list()
    
    pupilData_filter = list()
    time_filter = list()
    index_blinkEnd = list()
    
    missingBeginning = 0
    missingEnd = 0

    # First, find the missing values
    pupilInd = -1
    for pupilD in pupilData:
        
        pupilInd = pupilInd + 1
        #if pupilInd>772 and pupilInd<940:
        #    print(pupilInd, pupilD)
            
        # check if val is nan
        if np.isnan(pupilD):
            # check if first element of list
            if pupilInd == 0:
                # first value is nan
                missingPupilData['Start'].append(pupilInd)
                #print('Start', missingPupilData['Start'][-1])
                
            # check if previous val was nan
            elif np.isnan(pupilData[pupilInd-1]):
                
                # check if val is not last element
                if pupilInd < len(pupilData)-1:
                    # check if next val is not nan
                    if not np.isnan(pupilData[pupilInd+1]):
                        # note the index for end of blink
                        missingPupilData['End'].append(pupilInd)
                        #print('End', missingPupilData['End'][-1])
                else:
                    # last value is nan
                    missingEnd = 1
                    missingPupilData['End'].append(pupilInd)
                    #print('End', missingPupilData['End'][-1])
                    
            # if new nan, note the index
            else:
                missingPupilData['Start'].append(pupilInd)
                #print('Start', missingPupilData['Start'][-1])
                
                # for single nan values
                # check if val is not last element
                if pupilInd < len(pupilData)-1:
                    # check if next val is not nan
                    if not np.isnan(pupilData[pupilInd+1]):
                        # note the index for end of blink
                        missingPupilData['End'].append(pupilInd)
                        #print('End', missingPupilData['End'][-1])
                else:
                    # last value is nan
                    missingPupilData['End'].append(pupilInd)
                    #print('End', missingPupilData['End'][-1])
            
    
    
    # Second, create a list that does not contain missing values or 200ms before and after that
    
     # blink is every nan value in the range of 100-400ms 
    # 200 ms (18 samples) before and after the blink will also be removed
    addedBlinkSamples = 18    
    blinkDurationStart = 0
    blinkDurationEnd = 0
    
    iCurrent = len(missingPupilData['Start'])
    endCurrent = iCurrent - 1
    isBlink = 0
    pupilData_filter = list(pupilData)
    time_filter = list(timeListComplete)
    
    index_blinkEndExtra = list() # list to document if 2 blinks joined 
    
    while iCurrent > 0:
        
        iCurrent = iCurrent - 1
        
        #print('Current', missingPupilData['Start'][iCurrent], missingPupilData['End'][iCurrent])
        
        # check if it is really a blink, and remove if it is not
        # if blink duration is less than 100ms, the reason for missing data is something else
        timeStart0 = time_filter[missingPupilData['Start'][iCurrent]].partition('+')
        timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
        timeEnd0 = time_filter[missingPupilData['End'][iCurrent]].partition('+')
        timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
        time1 = (timeEnd - timeStart).total_seconds()
        #print(time1)
        if time1 < 0.1 or time1 > 1.5:
            isNotBlink = 1
        else:
            isNotBlink = 0
            
        # Remove if it is a blink
        if isNotBlink:
            del pupilData_filter[missingPupilData['Start'][iCurrent]:missingPupilData['End'][iCurrent]+1]
            del time_filter[missingPupilData['Start'][iCurrent]:missingPupilData['End'][iCurrent]+1]
            
            # length of non-blink
            lengthBlink = missingPupilData['End'][iCurrent] - missingPupilData['Start'][iCurrent] + 1
            
            # all blinks after this non-blink need to be reduced in indexes
            #print(len(missingPupilData['Start'])-1)
            
            for iChange in range(iCurrent-1, len(missingPupilData['Start'])-1):
                #print('i', iChange)
                if iChange == iCurrent-1:
                    
                    #print('Deleted', missingPupilData['Start'][iChange+1], missingPupilData['End'][iChange+1])
                    #print(lengthBlink)
                    del missingPupilData['Start'][iChange+1]
                    del missingPupilData['End'][iChange+1]
                else:
                    
                    #print('Old', missingPupilData['Start'][iChange], missingPupilData['End'][iChange])
                    missingPupilData['Start'][iChange] = missingPupilData['Start'][iChange] - lengthBlink
                    missingPupilData['End'][iChange] = missingPupilData['End'][iChange] - lengthBlink
                    #print('New', missingPupilData['Start'][iChange], missingPupilData['End'][iChange])
                    
    #print(len(missingPupilData['Start']))
    
    previousBlinkEnd = 0
    nextBlinkEnd = 0
    iCurrent = len(missingPupilData['Start'])
    
#     for index, (value1, value2) in enumerate(zip(missingPupilData['Start'], missingPupilData['End'])):
#         print(index, value1, value2)
#     for i in enumerate(pupilData_filter):
#         print(i)
    
    while iCurrent > 0:
        
        iCurrent = iCurrent - 1
        #print(iCurrent)
        iNow = iCurrent # iCurrent is modified in advance later (possibly), which will disturb the current data index
        #print(missingPupilData['Start'][iNow], missingPupilData['End'][iNow])
        
#         if iNow > 1 :
#             print('BEFORE!!!!!!!!!!!!!!!!!')
#             print(pupilData_filter[missingPupilData['Start'][iNow]:-1])
            
        # to make sure that blink removal data does not go into the previous data
        if iCurrent == 0:
            previousBlinkEnd = 0
        else:
            previousBlinkEnd = missingPupilData['Start'][iCurrent-1]

        # to make sure blink does not go into the next one
        if iCurrent == len(missingPupilData['Start'])-1:
            #print(iCurrent)
            nextBlinkEnd = len(pupilData_filter)
            #print(nextBlinkEnd)
        else:
            nextBlinkEnd = missingPupilData['Start'][iCurrent+1]
            
        
        
        if missingPupilData['End'][iCurrent] + addedBlinkSamples > nextBlinkEnd:
            #LOOK INTO THIS, NEXT BLINK IS REMOVED BUT ITS EXTRA 200MS IS NOT!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            #print('After', missingPupilData['End'][iCurrent], addedBlinkSamples, '>>>>>', nextBlinkEnd)
            blinkRemoveAfter = abs(missingPupilData['End'][iCurrent] - nextBlinkEnd)
        else:
            blinkRemoveAfter = addedBlinkSamples
        
        if missingPupilData['Start'][iCurrent] - addedBlinkSamples < previousBlinkEnd:
            #print('Before', missingPupilData['Start'][iCurrent], addedBlinkSamples, '<<<<<<', previousBlinkEnd)
            blinkRemoveBefore = abs(missingPupilData['Start'][iCurrent] - previousBlinkEnd)
            
            # Add the extra duration to list
            if iCurrent > 0:
                
                timeStart0 = time_filter[missingPupilData['Start'][iCurrent-1]].partition('+')
                timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                timeEnd0 = time_filter[missingPupilData['End'][iCurrent-1]].partition('+')
                timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                time1 = (timeEnd - timeStart).total_seconds()
                #print(timeEnd, timeStart, time1)

                missingPupilData['DurationExtra'].append(time1)
#             if iCurrent > 1:
#                 print(missingPupilData['Start'][iCurrent-1], 'also removed')
#             # After next blink is also removed, 
            iCurrent = iCurrent - 1
            index_blinkEndExtra.append(2)
            
                
        else:
            blinkRemoveBefore = addedBlinkSamples
            index_blinkEndExtra.append(1)
            missingPupilData['DurationExtra'].append(0)
            
#         print(missingPupilData['Start'][iNow]-blinkRemoveBefore)
        
        timeStart0 = time_filter[missingPupilData['Start'][iNow]].partition('+')
        timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        #print(timeStart, pupilData_filter[missingPupilData['Start'][iNow]])     
        timeEnd0 = time_filter[missingPupilData['End'][iNow]].partition('+')
        timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        #print(timeEnd) 
        time2 = (timeEnd - timeStart).total_seconds()
        #print(time2)
        missingPupilData['Duration'].append(time2)
        del pupilData_filter[missingPupilData['Start'][iNow]-blinkRemoveBefore:missingPupilData['End'][iNow]+blinkRemoveAfter]
        del time_filter[missingPupilData['Start'][iNow]-blinkRemoveBefore:missingPupilData['End'][iNow]+blinkRemoveAfter]
        
        index_blinkEnd.append(missingPupilData['Start'][iNow] - blinkRemoveBefore - 1)
    #         if iNow > 1 :
#             print('AFTER!!!!!!!!!!!!')
#             if np.nan in pupilData_filter[missingPupilData['Start'][iNow]:-1]:
#                 for i,j in enumerate(pupilData_filter[missingPupilData['Start'][iNow]:-1]):
#                     print(i+missingPupilData['End'][iNow]+blinkRemoveAfter,j)
        
    
    if np.nan in pupilData_filter:
        for i in enumerate(pupilData_filter):
            print(i)
     
    index_blinkEnd = list(reversed(index_blinkEnd))
    
    return pupilData_filter, time_filter, index_blinkEnd, index_blinkEndExtra, missingPupilData

In [4]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [5]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [13]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
flagFirstSubj = 0
ipaList = list()
blink = dict()
blink['rate_DT'] = list()
blink['rate_MS'] = list()

blink['duration_DT'] = list()
blink['duration_MS'] = list()

for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs:
        
        if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
            #notCompleted, or they were not native Danish speakers and are marked as notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        userKeys = None
        scratchPad = None
        gazeLog = None
        stimPhrase = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
                        
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
            else:
                continue
            
                # if all these lists exist
            if gazeLog is None or scratchPad is None:
                continue
            else:
                
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                subjName = a.findall(root)[0]
                typing_mechanism = subjName[-2:]
                print(subjName)
                
                # obtain right and left pupil data
                # Create list of pupil sizes from gazelog
                pupilLogL = [float(item4[29]) if 'Invalid' not in item4 else np.nan for item4 in gazeLog]
                pupilLogR = [float(item5[31]) if 'Invalid' not in item5 else np.nan for item5 in gazeLog]
                timeList = [item[0] for item in gazeLog]
                
                # filter the blinks
                pupilLogL_filter, time_filter, index_blinkEndL, index_blinkEndExtra, missingDataL = filterBlinksNew(pupilLogL, timeList)
                
                pupilLogR_filter, time_filter, index_blinkEndR, index_blinkEndExtra, missingDataR = filterBlinksNew(pupilLogR, timeList)
                
                
                
                time_filterFormatted = timeConversion(time_filter)
                pupilLog_avg = [(pupilLogL_filter[i] + pupilLogR_filter[i])/2 for i in range(0, len(pupilLogL_filter))]
                
                
                
#                 fig=plt.figure()
#                 ax = fig.add_subplot(111)
#                 ax.plot(pupilLog_avg)
#                 ylim =  [min(pupilLog_avg)-0.05, max(pupilLog_avg) + 0.05]
#                 for xPoint in index_blinkEnd:
#                     ax.plot([xPoint, xPoint], ylim, '--', 'k', alpha = 0.3)
#                 ax.set_ylim(ylim)
                
    
                # find end time of Expt - when the scratchpad is erased, at the end
                scratchPadText = [i[1] for i in scratchPad]
                scratchPadTime = [i[0] for i in scratchPad]
                
                iText = len(scratchPadText)
                while iText > 0:
                    iText = iText - 1
                    if scratchPadText[iText] == '':
                        if scratchPadText[iText-1] != '':
                            findTimeAtIndex = iText
                            break
                    
                timeExptEndStr = scratchPadTime[findTimeAtIndex]
                timeExptEndStr1, t1, t2 = timeExptEndStr.partition('+')
                timeExptEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeExptEndStr1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                timeExptStartStr = scratchPadTime[0]
                timeExptStartStr1, t1, t2 = timeExptStartStr.partition('+')
                timeExptStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeExptStartStr1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                # find the actual start of the fixation
                
                timePeriodExpt = ((timeExptEnd - timeExptStart).total_seconds())
                
                blinkDuration = np.mean(missingDataL['Duration'])/timePeriodExpt
                
                if 'DT' in subjName:
                    blink['rate_DT'].append(len(index_blinkEndL)*60/timePeriodExpt)
                    blink['duration_DT'].append(blinkDuration)
                else:
                    blink['rate_MS'].append(len(index_blinkEndL)*60/timePeriodExpt)
                    blink['duration_MS'].append(blinkDuration)
                
                
                print(len(index_blinkEndL), timeExptEnd - timeExptStart)
                


akt_MS
56 0:05:54.014577
jl_DT
50 0:05:22.388514
KEA_MS
10 0:02:45.356817
lone_DT
27 0:01:57.458496
mcc_MS
16 0:02:27.255528
MK_DT
33 0:06:04.175399
ok_MS
12 0:03:00.793634
pt_DT
37 0:05:14.864165
sc_MS
49 0:01:54.283541
sh_MS
46 0:05:26.977646
ae_DT
39 0:04:33.334904
hc_MS
72 0:08:59.308466
ib_MS
91 0:09:01.678976
km_DT
16 0:06:56.412204
ma_DT
50 0:05:08.960545
pgba_DT
2 0:05:04.825487
smn_DT
21 0:06:01.142914
eo_DT
42 0:03:18.260458
jek_MS
65 0:08:21.530594
jg_DT
19 0:04:16.334513
lg_MS
50 0:03:22.388047
lr_MS
6 0:03:59.605526
mm_MS
44 0:05:33.900052
snk_DT
41 0:06:06.933081


In [7]:
# Boxplot

fig = plt.figure()
ax = fig.add_subplot(1,1,1) 

ax.boxplot([blink['rate_DT'], blink['rate_MS']], positions= [1, 2])
xTicks = ax.get_xticks()
#print(xTicks)
ax.set_xticklabels(['DT', 'MS'])

ax.set_title('DT- mean: {0}, std: {1}                      MS - mean:  {2},  std:  {3}'.format(np.round(np.mean(blink['rate_DT']), 3), np.round(np.std(blink['rate_DT']),3), np.round(np.mean(blink['rate_MS']),3), np.round(np.std(blink['rate_MS']),3)))
#ax.set_ylim([0, 0.5])

print(scipy.stats.ttest_ind(blink['rate_DT'], blink['rate_MS']))
print(scipy.stats.ranksums(blink['rate_DT'], blink['rate_MS']))

print('DT- mean: {0}, std: {1}                      MS - mean:  {2},  std:  {3}'.format(np.round(np.mean(blink['rate_DT']), 3), np.round(np.std(blink['rate_DT']),3), np.round(np.mean(blink['rate_MS']),3), np.round(np.std(blink['rate_MS']),3)))

Ttest_indResult(statistic=-0.9247906861898595, pvalue=0.3651082299261139)
RanksumsResult(statistic=-0.6350852961085883, pvalue=0.5253727863737021)
DT- mean: 6.992, std: 3.883                      MS - mean:  8.991,  std:  6.024


In [9]:
# Boxplot

fig = plt.figure()
ax = fig.add_subplot(1,1,1) 

ax.boxplot([blink['duration_DT'], blink['duration_MS']], positions= [1, 2])
xTicks = ax.get_xticks()
#print(xTicks)
ax.set_xticklabels(['DT', 'MS'])

ax.set_title('DT- mean: {0}, std: {1}                      MS - mean:  {2},  std:  {3}'.format(np.round(np.mean(blink['duration_DT']), 5), np.round(np.std(blink['duration_DT']),5), np.round(np.mean(blink['duration_MS']),5), np.round(np.std(blink['duration_MS']),5)))
#ax.set_ylim([0, 0.5])

print(scipy.stats.ttest_ind(blink['duration_DT'], blink['duration_MS']))
print(scipy.stats.ranksums(blink['duration_DT'], blink['duration_MS']))

print('DT- mean: {0}, std: {1}                      MS - mean:  {2},  std:  {3}'.format(np.round(np.mean(blink['duration_DT']), 5), np.round(np.std(blink['duration_DT']),5), np.round(np.mean(blink['duration_MS']),5), np.round(np.std(blink['duration_MS']),5)))

Ttest_indResult(statistic=-0.891680539640423, pvalue=0.38221335290964664)
RanksumsResult(statistic=-0.8660254037844386, pvalue=0.3864762307712327)
DT- mean: 0.00217, std: 0.00152                      MS - mean:  0.00292,  std:  0.00232
